In [13]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [14]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [15]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,100,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,10,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,50,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,50,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,50,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,50,url)

In [16]:
# getting rid of non-ascii characters from the reviews as it breaks csv

df_recent.review_content = df_recent.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_new.review_content = df_new.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_fave.review_content = df_fave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_rant.review_content = df_rant.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_anom.review_content = df_anom.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [17]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [18]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



49767
46546


In [19]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
760,Evil Twin Even More Coco Jesus,2.3,-1.6,Evil Twin Brewing,"Can, early 2018. Soapy coconut, lots of molass...",ElDesmadre
761,Lösch-Zwerg Würzig,0.5,-2.2,Privatbrauerei Schimpfle,Genuinely couldn't finish this. Horribly astri...,LocalMuse25
762,Scheyern Kloster-Gold Hell,0.7,-2.1,Kloster-Brauerei Scheyern,"Horrid beer. Nasty aromas, even more nasty fla...",LocalMuse25
763,Squatters Hop Rising Tropical Double IPA,5.0,+1.6,Squatters Craft Beers (CANarchy Craft Brewery ...,"Sniff the aroma, look at the color, its the be...",beer3000


In [20]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [21]:
from selenium import webdriver
import time

In [22]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [23]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 1465 new beers to search for!


In [27]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [28]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

,return_string
0,Yachats First Responder Session IPA\n🇺🇸 ISA - ...
1,Yellow Dog Faded White IPA\n🇨🇦 IPA - White • 6...
2,Seapine Citra IPA\n🇺🇸 IPA • 6.6%\n-\n-\n3.43\n(6)
3,Twin Sails Vaporwave\n🇨🇦 IPA - Sour / Wild • 6...
4,Twin Sails Lil’ Gulp Radler\n🇨🇦 Radler / Shand...
...,...
1460,Meckatzer Fest-Märzen\n🇩🇪 Märzen / Marzen / Ok...
1461,Kizakura / Kyoto Bakushu Kölsch\n🇯🇵 Kölsch / K...
1462,Union Smile (2020-)\n🇸🇮 Pale Lager - North Am....
1463,Bruha New England IPA\n🇬🇧 IPA - Hazy / NEIPA •...


In [29]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [30]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [31]:
# need to add in add check here for price


In [32]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

,return_string,name_found,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number
35,Texas Ale Project Tap Wtr - Mixed Berry Seltze...,Texas Ale Project Tap Wtr - Mixed Berry Seltzer,🇺🇸,Flavored - Other,NaN,-,-,3.00,1
75,Belgh Brasse Morbach Kölsch\n🇨🇦 Kölsch / Kolsc...,Belgh Brasse Morbach Kölsch,🇨🇦,Kölsch / Kolsch-Style,NaN,-,-,3.17,3
89,Warped Wing Alebrije Gose\n🇺🇸 Gose\n-\n-\n3.50...,Warped Wing Alebrije Gose,🇺🇸,Gose,NaN,-,-,3.50,1
114,Wooden Cask There Gose Cutie\n🇺🇸 Gose - Flavor...,Wooden Cask There Gose Cutie,🇺🇸,Gose - Flavored,NaN,-,-,3.40,1
120,Rock House Flat Top Griddle\n🇺🇸 Porter - Flavo...,Rock House Flat Top Griddle,🇺🇸,Porter - Flavored,NaN,-,-,3.70,1
391,Rock House Domo Affogato\n🇺🇸 Porter - Flavored...,Rock House Domo Affogato,🇺🇸,Porter - Flavored,NaN,-,-,3.70,1
407,Roadhouse Pono Life\n🇺🇸 ISA - Session IPA\n-\n...,Roadhouse Pono Life,🇺🇸,ISA - Session IPA,NaN,-,-,2.00,1
482,Amstel 0.0\n🇳🇱 Low / No Alcohol Beer - Pale0\n...,Amstel 0.0,🇳🇱,Low / No Alcohol Beer - Pale,0.0,2,35,1.96,86
508,AND UNION Tuesday\n🇩🇪 Low / No Alcohol Beer - ...,AND UNION Tuesday,🇩🇪,Low / No Alcohol Beer - Wheat,NaN,-,-,2.50,1
555,Odd13 DDH Nelson Sauvin Codename: Superfan\n🇺🇸...,Odd13 DDH Nelson Sauvin Codename: Superfan,🇺🇸,IPA - Hazy / NEIPA,NaN,-,-,3.90,1


In [33]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [34]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
1161,Oskar Blues Saloon Juice\n🇺🇸 Amber Lager - Cze...,🇺🇸,Amber Lager - Czech / Polotmavý,NaN,-,-,2.50,2,Oskar Blues Saloon Juice
1190,Counterpart Flow\n🇨🇦 Pale Ale\n-\n-\n3.25\n(6),🇨🇦,Pale Ale,NaN,-,-,3.25,6,Counterpart Flow
1219,Rothaus Alkoholfrei Hefeweizen\n🇩🇪 Low / No Al...,🇩🇪,Low / No Alcohol Beer - Wheat,0.0,Available from £1.99,7,44,2.37,Rothaus Alkoholfrei Hefeweizen
1260,Épitaphe Vøringfossen\n🇨🇦 IPA\n-\n-\n3.87\n(7),🇨🇦,IPA,NaN,-,-,3.87,7,Épitaphe Vøringfossen


In [35]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 67 new beers that were not found or returned errors. They will be added to the no search list.


In [36]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)